## So what am I doing here?

So what are the steps:
    - Load the data
    - Set-up my machine learning environment
    - Organize the data into an appropriate directory structure (mimic what he has been doing) with training, validation and test sets (making the right validation set will possibly be tricky on this dataset
    - Read the data into the needed data structures for learning (I still really need to learn how to do this)
    - Start with a linear model
    - Then start doing multilayer MLP
    - Then start using the pre-trained network, pre-compute convolutional layers, train fully connected layers
    - Start applying the techniques that Jeremy Howard has been teaching.

## Step 1. Load the data

In [8]:
%pwd


lesson3/  utils/
downloading https://www.kaggle.com/c/state-farm-distracted-driver-detection/download/sample_submission.csv.zip

sample_submission.csv.zip N/A% |                      | ETA:  --:--:--   0.0 s/B

Is it possible you have not accepted the competition's rules on the kaggle website?



In [1]:
#Create references to important directories we will use over and over
import os, sys
current_dir = os.getcwd()
LESSON_HOME_DIR = current_dir
DATA_HOME_DIR = current_dir+'/data'

In [2]:
#Allow relative imports to directories above lesson1/
sys.path.insert(1, os.path.join(sys.path[0], '../utils/'))

#import modules
from utils import *; 
from vgg16 import Vgg16

#Instantiate plotting tool
#In Jupyter notebooks, you will need to run this command before doing any plotting
%matplotlib inline

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

WARNING (theano.sandbox.cuda): CUDA is installed, but device gpu is not available  (error: Unable to get the number of gpus available: unknown error)
Using Theano backend.


In [3]:
#Create directories
%cd $DATA_HOME_DIR
!kg download -u remi10001 -p $KAGGLE_PASS -c state-farm-distracted-driver-detection
% unzip -q imgs.zip
% rm imgs.zip
%mkdir valid
%mkdir results
%mkdir -p sample/train
%mkdir -p sample/test/unknown
%mkdir -p sample/valid
%mkdir -p sample/results
#%mkdir -p test/unknown
%mkdir tester
%mv test/ tester/unknown
%mv tester test

/home/ubuntu/nbs-git/state_farm_comp/lesson3/data


# Create the Validation set, train and validation separate drivers

In [3]:
%pwd

u'/home/ubuntu/nbs-git/state_farm_comp/lesson3'

In [4]:
# Assess class distribution for the subjects in the training dataset
%cd $DATA_HOME_DIR
import pandas as pd
# Read in the data table
a = pd.read_csv('driver_imgs_list.csv')

tab = a.groupby(['subject', 'classname']).size()
print(tab.unstack())
tab = a.groupby('subject').size()
print(tab)

# select three subjects at random for the validation set
import random
random.seed(100)   # So subjects selected are consistent
b =set(np.random.permutation(a['subject']))
subs_val = random.sample(b - set('p072'), 3)# Decided on 3 drivers with further consultation from Jeremy Howard's notebook
print("Validation subjects: " + ', '.join(subs_val))

a['val.file'] = a[['classname', 'img']].apply(lambda x: '/'.join(x), axis=1)
tab_val = a.loc[a['subject'].isin(subs_val)]
val_files =tab_val['val.file'].tolist()
val_files[0:2]

/home/ubuntu/nbs-git/state_farm_comp/lesson3/data
classname   c0   c1   c2   c3   c4   c5   c6   c7   c8   c9
subject                                                    
p002        76   74   86   79   84   76   83   72   44   51
p012        84   95   91   89   97   96   75   72   62   62
p014       100  103  100  100  103  102  101   77   38   52
p015        79   85   88   94  101  101   99   81   86   61
p016       111  102  101  128  104  104  108  101   99  120
p021       135  131  127  128  132  130  126   98   99  131
p022       129  129  128  129  130  130  131   98   98  131
p024       130  129  128  130  129  131  129  101   99  120
p026       130  129  130  131  126  130  128   97   97   98
p035        94   81   88   89   89   89   94   87   56   81
p039        65   63   70   65   62   64   63   64   70   65
p041        60   64   60   60   60   61   61   61   59   59
p042        59   59   60   59   58   59   59   59   59   60
p045        75   75   76   75   75   76   71   67 

['c0/img_68359.jpg', 'c0/img_35502.jpg']

In [5]:
# Create the separate directories in valid and sample/train
os.chdir(DATA_HOME_DIR+'/train')
for d in glob('c?'):
    os.mkdir('../sample/train/' + d)
    os.mkdir('../sample/valid/' + d)
    os.mkdir('../valid/' + d)

In [6]:
# Now move the files from the training folder to the validation folder
os.chdir(DATA_HOME_DIR + '/train')
from shutil import copyfile
for f in val_files:
    os.rename(f, DATA_HOME_DIR + '/valid/' + f)

In [7]:
# Now copy files to sample folders
#How many do I want in my sample? maybe 50 train, 20 valid, 20 test [just to make sure code is working]
train_sample = 50
valid_sample = 20
test_sample = 20

os.chdir(DATA_HOME_DIR + '/train')
fs = glob('c?/*.jpg')
shuf = np.random.permutation(fs)
for i in range(train_sample):
    copyfile(shuf[i], '../sample/train/' + shuf[i])

In [8]:
os.chdir(DATA_HOME_DIR + '/valid')
fs = glob('c?/*.jpg')
shuf = np.random.permutation(fs)
for i in range(valid_sample):
    copyfile(shuf[i], '../sample/valid/' + shuf[i]) 

In [9]:
os.chdir(DATA_HOME_DIR + '/test/unknown')
fs = glob('*.jpg')
shuf = np.random.permutation(fs)
for i in range(test_sample):
    copyfile(shuf[i], '../../sample/test/unknown/' + shuf[i]) 

So now assume I have the data in all the directories I want. Now what do I need to do? One approach is that I can make a simple linear model in keras. 
    - I need to get batches of my training data and validation data. I can use the get batches function for that. 
    - Then I can define a sequential linear model
    

In [3]:
%cd $DATA_HOME_DIR
#Set path to sample/ path if desired
path = DATA_HOME_DIR + '/sample/' #'/'

test_path = DATA_HOME_DIR + '/test/' #We use all the test data
results_path=DATA_HOME_DIR + '/results/'
train_path=path + '/train/'
valid_path=path + '/valid/'

/home/ubuntu/nbs-git/state_farm_comp/lesson3/data


In [31]:
batch_size = 64

In [5]:
train_batches = get_batches(train_path, batch_size=batch_size)
val_batches = get_batches(valid_path, batch_size=batch_size*2, shuffle=False)

Found 50 images belonging to 10 classes.
Found 20 images belonging to 10 classes.


In [6]:
??get_classes

In [7]:
(val_classes, trn_classes, val_labels, trn_labels, val_filenames, 
 train_filenames, test_filenames) = get_classes(path)

Found 50 images belonging to 10 classes.
Found 20 images belonging to 10 classes.
Found 20 images belonging to 1 classes.


In [10]:
model = Sequential([
    BatchNormalization(axis=1, input_shape=(3,224,224)),
    Flatten(),
    Dense(10, activation="softmax")
])

In [11]:
model.compile(optimizer="adam",
             loss="categorical_crossentropy",
             metrics=['accuracy'])

In [16]:
model.fit_generator(train_batches, train_batches.nb_sample, nb_epoch=10, 
                    validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

Epoch 1/10
50/50 [==============================] - 1s - loss: 9.7120 - acc: 0.1200 - val_loss: 15.3122 - val_acc: 0.0500
Epoch 2/10
50/50 [==============================] - 0s - loss: 10.9603 - acc: 0.3200 - val_loss: 15.3122 - val_acc: 0.0500
Epoch 3/10
50/50 [==============================] - 0s - loss: 11.2827 - acc: 0.3000 - val_loss: 15.3122 - val_acc: 0.0500
Epoch 4/10
50/50 [==============================] - 0s - loss: 11.2827 - acc: 0.3000 - val_loss: 15.3122 - val_acc: 0.0500
Epoch 5/10
50/50 [==============================] - 0s - loss: 11.2827 - acc: 0.3000 - val_loss: 15.3122 - val_acc: 0.0500
Epoch 6/10
50/50 [==============================] - 1s - loss: 11.2827 - acc: 0.3000 - val_loss: 15.3122 - val_acc: 0.0500
Epoch 7/10
50/50 [==============================] - 0s - loss: 11.2827 - acc: 0.3000 - val_loss: 15.3122 - val_acc: 0.0500
Epoch 8/10
50/50 [==============================] - 0s - loss: 11.2827 - acc: 0.3000 - val_loss: 15.3122 - val_acc: 0.0500
Epoch 9/10
50/50 

In [20]:
np.round(model.predict_generator(train_batches, train_batches.nb_sample)[:10], 2)

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.]], dtype=float32)

Nearly always predicting 1 of 2 classes. Decrease the learning rate to see if it will do better

In [21]:
model = Sequential([
    BatchNormalization(axis=1, input_shape=(3,224,224)),
    Flatten(),
    Dense(10, activation="softmax")
])

model.compile(optimizer=Adam(lr=1e-5),
             loss="categorical_crossentropy",
             metrics=['accuracy'])

model.fit_generator(train_batches, train_batches.nb_sample, nb_epoch=10, 
                    validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

Epoch 1/10
50/50 [==============================] - 1s - loss: 3.6615 - acc: 0.1600 - val_loss: 14.6681 - val_acc: 0.0500
Epoch 2/10
50/50 [==============================] - 1s - loss: 2.5600 - acc: 0.1800 - val_loss: 10.4166 - val_acc: 0.0500
Epoch 3/10
50/50 [==============================] - 0s - loss: 2.1767 - acc: 0.3600 - val_loss: 8.3791 - val_acc: 0.0500
Epoch 4/10
50/50 [==============================] - 0s - loss: 1.8068 - acc: 0.4600 - val_loss: 7.3171 - val_acc: 0.0500
Epoch 5/10
50/50 [==============================] - 0s - loss: 1.5889 - acc: 0.5000 - val_loss: 6.8109 - val_acc: 0.0500
Epoch 6/10
50/50 [==============================] - 0s - loss: 1.1698 - acc: 0.6400 - val_loss: 6.4534 - val_acc: 0.0500
Epoch 7/10
50/50 [==============================] - 0s - loss: 1.0534 - acc: 0.6600 - val_loss: 6.4669 - val_acc: 0.0500
Epoch 8/10
50/50 [==============================] - 0s - loss: 0.9361 - acc: 0.7200 - val_loss: 6.4526 - val_acc: 0.1000
Epoch 9/10
50/50 [============

In [22]:
3 * 224 * 224

150528

There are 150528 pixels. Thus, with 10 hidden units, the model can overfit the training data as much as it wants. That is what I see, ever increasing accuracy on the training set but no increase in accuracy on the validation set beyond chance (predicting the same class for all samples)

Let's go ahead straight to the Vgg16 pre-trained model and fine-tune it. If that takes too long, I can pre-compute the convolutional layers and then train the other layers. After I do this, I can go back and look through all the steps that he did.

In [23]:
vgg_model = Vgg16()
vgg_model.finetune(train_batches)
vgg_model.fit(train_batches, val_batches, nb_epoch=10)

Epoch 1/10
50/50 [==============================] - 22s - loss: 4.3720 - acc: 0.1400 - val_loss: 4.5390 - val_acc: 0.0500
Epoch 2/10
50/50 [==============================] - 21s - loss: 4.9361 - acc: 0.1400 - val_loss: 4.0187 - val_acc: 0.0500
Epoch 3/10
50/50 [==============================] - 21s - loss: 3.2543 - acc: 0.3600 - val_loss: 3.2506 - val_acc: 0.2500
Epoch 4/10
50/50 [==============================] - 21s - loss: 2.8068 - acc: 0.3200 - val_loss: 2.8694 - val_acc: 0.2000
Epoch 5/10
50/50 [==============================] - 21s - loss: 3.3810 - acc: 0.2200 - val_loss: 3.1717 - val_acc: 0.1500
Epoch 6/10
50/50 [==============================] - 21s - loss: 2.6494 - acc: 0.3000 - val_loss: 3.0018 - val_acc: 0.1500
Epoch 7/10
50/50 [==============================] - 21s - loss: 1.7781 - acc: 0.4400 - val_loss: 2.7893 - val_acc: 0.1500
Epoch 8/10
50/50 [==============================] - 21s - loss: 2.0502 - acc: 0.5000 - val_loss: 2.9055 - val_acc: 0.2000
Epoch 9/10
50/50 [======

The fine-tuned Vgg model on this amount of data can start to fit the training data and at least get to 20% validation accuracy.
I am using a vanishingly small amount of the data, so this is probably not a good test of these parameters, and pre-trained model architecture. IT IS EXCITING TO ME HOW SIMPLE THE VGG16 CODE NOW LOOKS. It is starting to make a lot of sense.

I think now I should start going through Jeremy Howard's approach. Try to write the code myself after looking at his, or especially just after considering his thoughts. I am working towards having my own thoughts on how to improve accuracy and being able to implement those thoughts through keras/theano directly, rather than through Jeremy's code. I can't rely on him as a crutch, though he is certainly helping me to learn things faster.

## My approach to good performance

I am going to recreate what Jeremy Howard did in his State Farm submission, and I will build on it. I will use the pre-trained Vgg16 network, with data augmentation, and dropout and batch normalization on the dense layers. I will use the same network that he used, basically. I will train this model with sequential lowering of learning rate on the validation set. Once performance stabilizes I will PREDICT ON THE TEST SET. I am going to do a 5x larger training set with data augmentation. And I will do a 2x larger test set with data augmentation after I have my predictions. Then I will continue to refine the model that I used (with the saved weights from the first time around). I should be able to get all of this code working on the CPU before I do it on the GPU, simply using my samples. The experiments won't be of any consequence.

At each step I need to save weights as necessary.

In [8]:
vgg = Vgg16()
model = vgg.model

In [24]:
last_conv_idx = [i for i,l in enumerate(model.layers) if type(l) is Convolution2D][-1]
conv_layers = model.layers[:last_conv_idx+1]

In [27]:
conv_model = Sequential(conv_layers)
train_batches = get_batches(train_path, batch_size=batch_size, shuffle=False)
val_batches = get_batches(valid_path, batch_size=batch_size*2, shuffle=False)
test_batches = get_batches(test_path, batch_size=batch_size, shuffle=False)
conv_feat = conv_model.predict_generator(train_batches, train_batches.nb_sample)
conv_val_feat = conv_model.predict_generator(val_batches, val_batches.nb_sample)
conv_test_feat = conv_model.predict_generator(test_batches, test_batches.nb_sample)

In [ ]:
save_array(path + "results/conv_feat.dat", conv_feat)
save_array(path + "results/conv_val_feat.dat", conv_val_feat)
save_array(path + "results/conv_test_feat.dat"), conv_test_feat)

In [29]:
conv_feat = load_array(path + "results/conv_feat.dat")
conv_test_feat = load_array(path + "results/conv_test_feat.dat")

In [30]:
??bcolz.carray

In [34]:
def get_my_layers(p=0.8):
    return [
        MaxPooling2D(input_shape = conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dropout(p),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(10, activation='softmax')
        ]
            
        

(512, 14, 14)

Pre-computed data augmentation + dropout
line 107 in statefarm, that's where I am at
gen_t